In [711]:
# Might need to dockerize later

# %pip install pandas
# %pip install numpy
# %pip install translate
# %pip install nltk
# %pip install openpyxl

In [712]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
import csv
csv_filename = "root_words.csv"
from translate import Translator
translator = Translator(from_lang="bn",to_lang="en")
import re
import locale


In [713]:
def LCstring(X, Y):
    m=len(X)
    n=len(Y)
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)]
    result = 0
    for i in range(m + 1):
        for j in range(n + 1):
            if (i == 0 or j == 0):
                LCSuff[i][j] = 0
            elif (X[i-1] == Y[j-1]):
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j])
            else:
                LCSuff[i][j] = 0
    return result

In [714]:
def LCsequence(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)

    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]

    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
    return L[m][n]


In [715]:
#fetch suffix
suffix=pd.read_excel('suffix_set.xlsx')
class1_suffix=suffix[suffix['weight']==1]
class2_suffix=suffix[suffix['weight']==2]
class3_suffix=suffix[suffix['weight'].isin([1,3])]
# hybrid_suffix=suffix[suffix['weight'].isin([3,4])]

In [716]:
dictnary = pd.read_excel('root_words.xlsx')
# display the first column
def slice_dictnary(test_word):
    selector=1
    if(test_word[0]=='উ'):
        prefix1=test_word[:selector]
        prefix2='ও'+test_word[1:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix1}+'))|(dictnary['word'].str.match(f'{prefix2}+'))]
    elif(test_word[0]=='এ'):
        prefix1=test_word[:selector]
        prefix2='আ'+test_word[1:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix1}+'))|(dictnary['word'].str.match(f'{prefix2}+'))]
    elif(test_word[0]=='ঐ'):
        prefix1=test_word[:selector]
        prefix2='ই'+test_word[1:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix1}+'))|(dictnary['word'].str.match(f'{prefix2}+'))]
    else:
        prefix=test_word[:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix}+'))]
    return(sub_dictnary)

In [717]:
def checking(test_word):
    global prediction_list
    global sub_dictnary
    if sub_dictnary['word'].isin([test_word]).any():
        prediction_list.append(test_word)
    else:
        return(True)

In [718]:
def checking_with_translation(left_context,test_word,right_context):
    root=[0.2,0.0,'']
    translation_word=translator.translate(test_word).lower()
    actual_window=left_context+" "+test_word+" "+right_context
    translation_window=translator.translate(actual_window).lower()
    while not root[2]:
        for predicted_word in prediction_list:
            error=(len(test_word)+len(predicted_word)-2*LCsequence(test_word,predicted_word))/(len(test_word)+len(predicted_word))
            if error<=root[0]: 
                test_window=left_context+" "+predicted_word+" "+right_context
                translation_testwindow=translator.translate(test_window).lower()
                accurecy=LCsequence(translation_window,translation_testwindow)/len(translation_testwindow)
#               print(translation_testwindow,accurecy)
                if accurecy>root[1]:
                    root[0]=error
                    root[1]=accurecy
                    root[2]=predicted_word
                elif accurecy==root[1]:
                    if(LCsequence(test_word,root[2])<=LCsequence(test_word,predicted_word)):
                        root[0]=error
                        root[1]=accurecy
                        root[2]=predicted_word
        if not root[2]:
            root[0]+=0.1
                
    return(root[2])


In [719]:
def predict_root(target_suffix_set,test_word):
    predicted_root=[test_word+suffix for suffix in target_suffix_set.suffix]
    for i in predicted_root:
        if checking(i):
            continue
    return predicted_root

In [720]:
def remove_suffix(target_suffix,word):
    final_suffix=''
    #find actual suffix 
    #find longest suffix bettwen target_suffix that belongs at the end of given word 
    SL_word=[]
    for i in target_suffix:
        count=0
        for j in range(-1,(-(len(i)+1)),-1):
            if (word[j]==i[j]):
                
                count+=1
            else:
                break
        if(count==len(i)):
            final_suffix=i
            SL_word.append(word.rstrip(final_suffix))
    return(SL_word)

In [721]:
def accuracy_calculator(outputs):
    count=0
    # actual_roots=['অক্ষর', 'জ্ঞান', 'যে', 'নেই', 'তারা', 'নিরক্ষর', 'বলা', 'যেতে', 'পরা', 'নিরক্ষর', 'মানুষ', 'জীবন', 'সবদিক', 'থাকা', 'পিছন', 'নিরক্ষর', 'বহু', 'ক্ষেত্র', 'মানুষ', 'সার্বিক', 'আত্মা', 'বিকাশ', 'অন্তর', 'নিরক্ষর', 'অভিশাপ', 'অভিশপ্ত', 'ব্যক্তি', 'সাধারণ', 'নিজ', 'অথবা', 'সমাজ', 'জন্য', 'কিছু', 'কে', 'দেখা', 'পারা', 'নিরক্ষর', 'পরিণত', 'হওয়া', 'নিরব', 'অনুসরণ', 'করা', 'অর্থাৎ', 'প্রত্যেক', 'ক্ষেত্র', 'পরিচয়', 'পড়া', 'নিরক্ষর', 'মানুষ', 'পৃথিবী', 'নিজ', 'পরিবেশ', 'লাঞ্ছনা', 'শোষণ', 'জানা', 'পারা', 'রবীন্দ্রনাথ', 'ঠাকুর', 'বলা', 'মানুষ', 'অন্ধত্ব', 'মত', 'নিরক্ষর', 'এই', 'দুর্ভাগ্য', 'দেশ', 'হতভাগ্য', 'জনগণ', 'সর্বাপেক্ষা', 'নিষ্ঠুর', 'অভিশাপ', 'নিরক্ষর', 'স্বরূপ', 'বোঝা', 'জন্য', 'আমি', 'আমি', 'দেশ', 'ভারত', 'বাহুল্য', 'চল', 'যখন', 'মোট', 'জনসংখ্যা', 'একতা', 'নিরক্ষর', 'এই', 'নিরক্ষর', 'দূর', 'করা', 'একান্ত', 'প্রয়োজন']
    actual_roots=['দেশ', 'সমাজসেবা', 'ক্ষেত্র', 'যেমন', 'বহুবিস্তৃত', 'সমস্যা', 'যেমন', 'বহু', 'মুখ', 'দেশ', 'বিরাট', 'ছাত্রসমাজ', 'তেমন', 'বিভিন্ন', 'দল', 'বিভক্ত', 'হওয়া', 'শহর', 'গ্রাম', 'নানাপ্রকার', 'সেবামূলক', 'কাজ', 'অংশগ্রহণ', 'করা', 'পরা', 'যে', 'নিরক্ষর', 'উন্নয়নশীল', 'ভারতবর্ষ', 'সবচেয়ে', 'বড়', 'অভিশাপ', 'সমাজসেবা', 'অঙ্গরূপ', 'সেই', 'নিরক্ষর', 'দূরীকরণ', 'ছাত্র', 'দের', 'প্রথম', 'প্রধান', 'কর্তব্য', 'পল্লী', 'প্রধান', 'এই', 'দেশ', 'দ্বিতীয়', 'বৃহত্তম', 'সমস্যা', 'পল্লীসংস্কার', 'সুতরাং', 'সমাজসেবা', 'বিষয়', 'রূপ', 'ছাত্র', 'দের', 'পল্লী', 'সংস্কার', 'অংশগ্রহণ', 'করা', 'হওয়া', 'পল্লী', 'উন্নয়ন', 'ছাত্র', 'দের', 'কাজ', 'হওয়া', 'পল্লী', 'পথঘাট', 'নির্মাণজীর্ণ', 'পুকুর', 'সংস্কার', 'সাধন', 'পরা', 'জল', 'ব্যবস্থা', 'করা', 'ইত্যাদি', 'তাছাড়া', 'অজ্ঞ', 'উদাসীন', 'গ্রামবাসী', 'স্বাস্থ্যবিধি', 'পালন', 'সচেতন', 'করে', 'তোলা', 'ক্ষতিকর', 'সংস্কার', 'পরিহার', 'উৎসাহ', 'করা', 'তারা', 'আচরণ', 'নাগরিক', 'বোধ', 'গড়া', 'তোলা', 'সমাজসেবা', 'অন্তর্গত']
    for root_index in range(len(outputs)):
        
        if outputs[root_index] == actual_roots[root_index]:
            count+=1
    print("accuracy is ",(count/len(actual_roots))*100,'%')

In [722]:
def add_to_dataset(context_word, root_word):
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        rows = [row for row in reader]
        # Find the row with the root word and add the related word to it
        found_root_word = False
        for row in rows:
            if row[0] == root_word and context_word not in row:
                row.append(context_word)
                found_root_word = True
                break

        # Write the modified list back to the CSV file
        if found_root_word:
            with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(rows)
                print(f"Added {context_word} as a related word for {root_word}")
        else:
            with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                List=[[root_word,context_word]]
                writer.writerows(List)
                print(f"{root_word} inserted in csv")
                print(f"Added {context_word} as a related word for {root_word}")
                csvfile.close()

In [723]:
def csvAutoSorting():
    # set locale to Bengali
    locale.setlocale(locale.LC_ALL, 'bn_BD.UTF-8')

    # read words from CSV file
    words = []
    with open('root_words.csv', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            # get the first word from each row and append it to the list
            first_word = row[0].split()[0] # split the row by space and take the first word
            words.append((first_word, row))

    # sort words based on the first word only and write to new CSV file
    sorted_words = sorted(words, key=lambda x: locale.strxfrm(x[0]))
    with open('root_words.csv', mode='w', encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for word in sorted_words:
            writer.writerow(word[1])

In [724]:
def leematizer(context_word):
    global prediction_list
    container=[]
    #print(sub_dictnary)
    container.append(context_word)
    print("input ",context_word)
    if(context_word[0]=='উ'):
        container.append('ও'+context_word[1:])
        #print(container)
    elif(context_word[0]=='এ'):
        container.append('আ'+context_word[1:])
    elif(context_word[0]=='ঐ'):
        container.append('ই'+context_word[1:])
    for word in container:
        checking(word)
        #checking for direct root
        # if (class1_suffix['suffix'].isin([word[-1]]).any()):
        #     #if there is any suffix of class 1 remove it
        #     SL_word=remove_suffix(word[-1],word)
        #     # SL_word denoted suffix_less_word
        #     checking(SL_word)
        #     predict_root(class1_suffix,SL_word)
        index=[i in word for i  in class3_suffix.suffix]
        # select class 3 suffix
        if any(index):
            # sort target suffix list in desc order of length
            target_suffix=class3_suffix[index].suffix.tolist()
            target_suffix.sort(key=len,reverse=True)
            #print(target_suffix)
            SL_word_set=remove_suffix(target_suffix,word)
            if SL_word_set:
                #remove suffix
                #SL_word denoted suffix_less_word
                for SL_word in SL_word_set:
                    checking(SL_word)
                    predict_root(class2_suffix,word) # select only class 1 suffix suffix that is in the word
                    index=[]
                    #print(SL_word)
                    if len(SL_word)>1 and (class1_suffix['suffix'].isin([SL_word[-1]]).any()): #remove class 1 suffix suffix from end
                        S_word=SL_word.rstrip(SL_word[-1])
                    else:
                        S_word=SL_word
                    checking(S_word)
                    #if Sl_word is root word declare it or checck it with other suffix one by one
                    predict_root(class2_suffix,S_word)
                    predicted_subword=predict_root(class1_suffix,S_word)
                    #print(predicted_subword)
                    #if it is a root word declear rootword else check is there any other suffix present
                    #container.append(word)
                    for i in predicted_subword:
                        predict_root(class2_suffix,i)
    prediction_list=[*set(prediction_list)]
    #remove duplicate prediction 
    #print(prediction_list)
    if(len(prediction_list)==1): #if predicttion list contain only one prediction declear as root
        print(context_word,'--->',prediction_list[0])       
        add_to_dataset(context_word, prediction_list[0])
        return(prediction_list[0])
    elif(len(prediction_list)>1): #if more than one check using meaning
        root=checking_with_translation(left_context,context_word,right_context)
        print(context_word,'--->',root)
        add_to_dataset(context_word, root)
        return(root)
    else:
        print(context_word,"not in database")
        # root=[0,'']
        # for dict_word in sub_dictnary.word:
        #     lcs=LCsequence(dict_word,context_word)
        #     if(lcs>root[0]):
        #         root[0]=lcs
        #         root[1]=dict_word
        # print(context_word,'--->',root[1])
        # add_to_dataset(context_word, root[1])
        # return(root[1])
            


In [725]:
tokens=[]
#sentence1="ছাত্রজীবন মানব জীবনের প্রস্তুতিকাল। ছাত্রজীবনের ভিত্তির উপর ধীরে ধীরে মানবজীবনের কর্ম কৃতিত্বের সৌধটি গড়ে ওঠে। এই ভিত্তি সুদৃঢ় হবে, ভবিষ্যতে জীবন ও তত বলিষ্ঠ এবং সমৃদ্ধ হবে। মানব জীবনের ভিত্তি সুদৃঢ় করতে হলে ছাত্রজীবনে যত্নবান হতে হবে।তাই ছাত্রজীবনে লেখাপড়াও খেলাধুলার সঙ্গে সঙ্গে যেমন শরীর ও চরিত্র গঠন করতে হবে, তেমনি জগৎ ও জীবনের বহু বিচিত্র কর্মধারার সঙ্গে পরিচয় সাধন করতে হবে। এই পরিচয় সাধনের সূত্রই ছাত্রজীবনে সমাজকল্যাণ অথবা সমাজসেবা।"
#sentence2="সভ্যতার প্রথম উন্মেষ কালে আগুন আবিষ্কারের পরই সমাজ গঠন বোধকরি মানব সভ্যতার অন্যতম অবদান। তারপর থেকে সভ্যতার অগ্রগতি ও ক্রমবিকাশের সঙ্গে সঙ্গে গোষ্ঠীবদ্ধ মানুষ কতগুলি অলিখিত নিয়ম কানুন ও রীতিনীতি সূত্রে একে অন্যের সঙ্গে পরস্পর গাঁতা হয়ে পৃথিবীর এক এক অঞ্চলে একত্রে বসবাস করতে আরম্ভ করলো। এই ভাবেই একদিন সুসভ্য মানুষের সমাজ গড়ে উঠলো। সমাজের উদ্ভবের প্রথম দিন থেকেই মানুষ সমাজ সেবায় প্রয়োজন অনুভব করে। আজ সভ্যতার চরম উৎকর্ষের দিনেও মানব সমাজে এই সেবা ও সহযোগিতার প্রয়োজন এতটুকু কমেনি। সুতরাং সমাজসেবা বলতে সমাজের অন্তর্গত মানুষের সেবা ও কল্যাণকেই বোঝায়।"
#sentence3="সমাজসেবার প্রয়োজন সব দেশে থাকলেও ভারতবর্ষের মতো দরিদ্র উন্নয়নশীল বিশাল দেশে এর গুরুত্ব সমাধি। ১০০ কোটি জন অধ্যুষিত এই বিশাল দেশের কোটি কোটি মানুষ নিদারুণ দুঃখ দুর্দশা ও অভাব-অনটনে নিপীড়িত। দেশের অনেক মানুষই আজও নিরক্ষরতার নিবিড় অন্ধকারে নিমজ্জিত। এর ওপর আছে প্রাকৃতিক বিপর্যয় ঝড়-ঝঞ্ঝা খরা, বন্যা, ভূমিকম্প, ও দুর্ভিক্ষ মহামারীর তান্ডব লীলা। বলা বাহুল্য, এদেশে এসব সমস্যার ও দুর্যোগ এর মোকাবিলা করার প্রাথমিক দায়িত্ব দেশের সরকারের বিশেষত আমাদের দেশে যখন জনকল্যাণমুখী গণতান্ত্রিক রাষ্ট্র প্রতিষ্ঠিত। কিন্তু কী অর্থে ,কি সামর্থ্যে এই বিরাট সমস্যার সমাধান করা রাষ্ট্রশক্তির একার পক্ষে কখনোই সম্ভব নয়। সেই দিক দিয়ে এদেশের সমাজ সেবার গুরুত্ব অপরিসীম।"
#sentence4= "সুপ্রাচীন কাল থেকেই ভারতীয় উপমহাদেশ অর্থনৈতিক সমৃদ্ধি ও সাংস্কৃতিক ঐতিহ্যের জন্য সুপরিচিত। ঐতিহাসিক সিন্ধু সভ্যতা এই অঞ্চলেই গড়ে উঠেছিল। ইতিহাসের বিভিন্ন পর্বে এখানেই স্থাপিত হয়েছিল বিশালাকার একাধিক সাম্রাজ্য। নানা ইতিহাস-প্রসিদ্ধ বাণিজ্যপথ এই অঞ্চলের সঙ্গে বিশ্বের অন্যান্য সভ্যতার বাণিজ্যিক ও সাংস্কৃতিক সম্পর্ক রক্ষা করত। হিন্দু, বৌদ্ধ, জৈন ও শিখ বিশ্বের এই চার ধর্মের উৎসভূমি ভারত। খ্রিস্টীয় প্রথম সহস্রাব্দে জরথুস্ত্রীয় ধর্ম (পারসি ধর্ম), ইহুদি ধর্ম, খ্রিস্টধর্ম ও ইসলাম ধর্ম এ দেশে প্রবেশ করে, ও ভারতীয় সংস্কৃতিতে বিশেষ প্রভাব বিস্তার করে। অষ্টাদশ শতাব্দীর প্রথমার্ধ থেকে ব্রিটিশ ইস্ট ইন্ডিয়া কোম্পানি ধীরে ধীরে ভারতীয় ভূখণ্ডের অধিকাংশ অঞ্চল নিজেদের শাসনাধীনে আনতে সক্ষম হয়। উনবিংশ শতাব্দীর মধ্যভাগে এই দেশ পুরোদস্তুর একটি ব্রিটিশ উপনিবেশে পরিণত হয়ে ওঠে। অতঃপর, এক সুদীর্ঘ স্বাধীনতা সংগ্রামের মধ্য দিয়ে ১৯৪৭ সালে ভারত একটি স্বতন্ত্র রাষ্ট্ররূপে আত্মপ্রকাশ করে।১৯৫০ সালে সংবিধান প্রণয়নের মাধ্যমে ভারত একটি সার্বভৌম গণতান্ত্রিক প্রজাতন্ত্রে পরিণত হয়।"
#sentence5="এদেশের সমাজসেবার সমস্যাটির যেমন গরিষ্ঠ তেমনি এর ক্ষেত্রটি ও বহুদূর বিস্তৃত। এই বিশাল ক্ষেত্রে সমাজ সেবায় অংশগ্রহণ করবে কে বা কারা? অবশ্য সরকারের ভূমিকায় এতে অগ্রগণ্য ,কিন্তু দেশের অসংখ্য জনহিতকর সেবা প্রতিষ্ঠান গুলির ভূমিকা ও এতে নগণ্য নয়। সমাজসেবামূলক এই প্রতিষ্ঠান গুলির মধ্যে রামকৃষ্ণ মিশন, ভারত সেবাশ্রম, মাড়োয়ারি রিলিফা সোসাইটি, রেড ক্রস, সেন্ট জনস্ অ্যাম্বুলেন্স প্রভৃতি উল্লেখযোগ্য এবং তাদের অবদান তাদের অকুণ্ঠ ও নিরলস জনসেবা প্রশংসনীয়। তবে সমাজসেবার বিশাল ক্ষেত্র ও বিপুল গুরুত্বের তুলনায় এই সেবা উল্লেখযোগ্য হলেও পর্যাপ্ত নয়।সমাজসেবার দায়িত্ব পালন ও গুরু ভার বহনের জন্য তাই প্রয়োজন বৃহত্তর শক্তির।"
#sentence6="পাঠিকা ঠাকুরানি নিজ গুণে ক্ষমা করবেন। মাতালের গল্প ছাড়া আমার গতি নেই।"
sentence7="তাই শুধু স্বাভাবিকভাবে নয়, প্রয়োজনের কারণে ও দেশের সুবিশাল ছাত্রসমাজের সুবিপুল কর্মশক্তির ওপর সমাজসেবার অংশগ্রহণের কথা এসে পড়ে। এখানে ও সমাজ সেবায় ছাত্র দের সম্পর্ক। ছাত্রসমাজ শুধু শক্তি মান নয়,সমাজসেবার কাছে তারা যোগ্য ও উপযুক্ত ও বটে। কারণ সমাজ সেবার জন্য যে সময় এবং শ্রম এর প্রয়োজন, আগ্রহ থাকলেও নানান সমস্যায় নিত্য বিব্রত কর্মজীবী সংসারী মানুষ তা দিতে পারে না। আপাতত দৃষ্টিতে অসম্ভব হলেও দেশের ছাত্রসমাজই সমাজসেবার মহান দায়িত্ব পালনে সময় ও শ্রম নিয়োগ করতে সক্ষম।অসীম প্রাণশক্তিতে উদ্বেল, অফুরন্ত উৎসাহে চঞ্চল, উচ্ছৃসিত আনন্দে ভরপুর তাজা তরুণ ছাত্র সমাজ তাই যেকোনো দুঃসাহসী কাজে নিঃস্বার্থ পরোপকারিতায় ঝাঁপিয়ে পড়ার জন্য সদাই উন্মুখ। লেখাপড়া ও খেলাধুলা করেও ছাত্রদের যে উৎসাহ উদ্বৃত থাকে তাকেই সমাজসেবার কাজে নিয়োজিত করতে হবে। ছাত্রজীবনে সমাজসেবার এই গুরুত্ব ও প্রয়োজনীয়তার কথা বিবেচনা করেই আমাদের মাধ্যমিক স্তরের শিক্ষাক্রমে সমাজসেবাকে পাঠ্য ও আচরণীয় বিষয় হিসেবে গ্রহণ করা হয়েছে"
sentence8="অক্ষর জ্ঞান যার নেই তাকেই নিরক্ষর বলা যেতে পারে। নিরক্ষর মানুষ জীবনের সবদিক থেকেই পিছিয়ে নিরক্ষরতা বহু ক্ষেত্রে মানুষের সার্বিক আত্মিক বিকাশের অন্তরায়। নিরক্ষরতার অভিশাপে অভিশপ্ত ব্যক্তি সাধারণত নিজের অথবা সমাজের জন্য কিছু করে দেখাতে পারেনা। নিরক্ষরতার পরিণতি হলো নিরবতার অনুসরণ করা অর্থাৎ প্রত্যেক ক্ষেত্রে পিছিয়ে পড়া। নিরক্ষর মানুষ পৃথিবীকে নিজেকে পরিবেশকে লাঞ্ছনাকে ও শোষণকে জানতে পারেনা। রবীন্দ্রনাথ ঠাকুর বলেছেন মানুষের অন্ধত্বের মত নিরক্ষরতা এই দুর্ভাগ্য দেশের হতভাগ্য জনগণের সর্বাপেক্ষা নিষ্ঠুরতম অভিশাপ। নিরক্ষরতার স্বরূপকে বোঝার জন্য আমরা আমাদের দেশ ভারতকে বুঝলেই চলবে- যেখানে মোট জনসংখ্যার এক-তৃতীয়াংশ নিরক্ষর। এই নিরক্ষতাকে দূর করা একান্ত প্রয়োজনীয়।"
sentence9="দেশের সমাজসেবার ক্ষেত্রটি যেমন বহুবিস্তৃত, সমস্যাগুলি যেমন বহু মুখী, দেশের বিরাট ছাত্রসমাজ ও তেমনি বিভিন্ন দলে বিভক্ত হয়ে শহর ও গ্রামের নানাপ্রকার সেবামূলক কাজে অংশগ্রহণ করতে পারে। যে নিরক্ষরতা উন্নয়নশীল ভারতবর্ষের সবচেয়ে বড় অভিশাপ, সমাজসেবার অঙ্গরূপে সেই নিরক্ষরতা দূরীকরণে ছাত্র দের প্রথম ও প্রধান কর্তব্য। পল্লী প্রধান এই দেশের দ্বিতীয় বৃহত্তম সমস্যা পল্লীসংস্কার। সুতরাং সমাজসেবায় বিষয় রূপে ছাত্র দের পল্লী সংস্কারে অংশগ্রহণ করতে হবে। পল্লী উন্নয়ন ছাত্র দের কাজ হবে পল্লীর পথঘাট নির্মাণ,জীর্ণ পুকুরের সংস্কার সাধন, পানীয় জলের ব্যবস্থা করা ইত্যাদি। তাছাড়াও অজ্ঞ ও উদাসীন গ্রামবাসীদের স্বাস্থ্যবিধি পালনে সচেতন করে তোলা, ক্ষতিকর সংস্কার পরিহারে উৎসাহিত করা, তাদের আচরণে নাগরিকতা বোধ গড়ে তোলা সমাজসেবার অন্তর্গত।"

sentences=sentence9

In [726]:
hyphen_removed = sentences.replace("-"," ")
words_unprocessed=hyphen_removed.split(" ")

# Define a regular expression 
bengali_pattern = re.compile(r"[a-zA-Z0-9০১২৩৪৫৬৭৮৯।(),–.’\u200c/`‘'-;%]")

# Remove Bengali digits, english alphabets, digits, punctuations from the tokens
words = np.array([bengali_pattern.sub("", token) for token in words_unprocessed])
words=list(words[[len(word)>=2 for word in words]])
print(words)

['দেশের', 'সমাজসেবার', 'ক্ষেত্রটি', 'যেমন', 'বহুবিস্তৃত', 'সমস্যাগুলি', 'যেমন', 'বহু', 'মুখী', 'দেশের', 'বিরাট', 'ছাত্রসমাজ', 'তেমনি', 'বিভিন্ন', 'দলে', 'বিভক্ত', 'হয়ে', 'শহর', 'গ্রামের', 'নানাপ্রকার', 'সেবামূলক', 'কাজে', 'অংশগ্রহণ', 'করতে', 'পারে', 'যে', 'নিরক্ষরতা', 'উন্নয়নশীল', 'ভারতবর্ষের', 'সবচেয়ে', 'বড়', 'অভিশাপ', 'সমাজসেবার', 'অঙ্গরূপে', 'সেই', 'নিরক্ষরতা', 'দূরীকরণে', 'ছাত্র', 'দের', 'প্রথম', 'প্রধান', 'কর্তব্য', 'পল্লী', 'প্রধান', 'এই', 'দেশের', 'দ্বিতীয়', 'বৃহত্তম', 'সমস্যা', 'পল্লীসংস্কার', 'সুতরাং', 'সমাজসেবায়', 'বিষয়', 'রূপে', 'ছাত্র', 'দের', 'পল্লী', 'সংস্কারে', 'অংশগ্রহণ', 'করতে', 'হবে', 'পল্লী', 'উন্নয়ন', 'ছাত্র', 'দের', 'কাজ', 'হবে', 'পল্লীর', 'পথঘাট', 'নির্মাণজীর্ণ', 'পুকুরের', 'সংস্কার', 'সাধন', 'পানীয়', 'জলের', 'ব্যবস্থা', 'করা', 'ইত্যাদি', 'তাছাড়াও', 'অজ্ঞ', 'উদাসীন', 'গ্রামবাসীদের', 'স্বাস্থ্যবিধি', 'পালনে', 'সচেতন', 'করে', 'তোলা', 'ক্ষতিকর', 'সংস্কার', 'পরিহারে', 'উৎসাহিত', 'করা', 'তাদের', 'আচরণে', 'নাগরিকতা', 'বোধ', 'গড়ে', 'তোলা', 'সমাজসেবার', 'অন্তর্গত

In [727]:

counter=0;
output_roots=[]
for word in words:
    search_word = word
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        found_root_word = False
        #check if the word exists in dataset
        for row in reader:
            if len(row) > 0 and row[0] == search_word:
                print(f"{search_word} -> {row[0]}")
                found_root_word = True
                output_roots.append(row[0])
            elif len(row) > 0 and search_word in row:
                print(f"{search_word} -> {row[0]}")
                found_root_word = True
                output_roots.append(row[0])
                
        if not found_root_word:

            if words.index(word)==0:
                left_context=''
                right_context=words[words.index(word)+1]
            elif words.index(word)==len(words)-1:
                left_context=words[words.index(word)-1]
                right_context=''
            else:
                left_context=words[words.index(word)-1]
                right_context=words[words.index(word)+1]
            sub_dictnary=slice_dictnary(word)
            if not sub_dictnary.empty:
                prediction_list=[]
                output_roots.append(leematizer(word))  
            else :
                print (word,"not in database")
print(output_roots)
accuracy_calculator(output_roots)

csvAutoSorting()     

দেশের -> দেশ
সমাজসেবার -> সমাজসেবা
ক্ষেত্রটি -> ক্ষেত্র
যেমন -> যেমন
বহুবিস্তৃত -> বহুবিস্তৃত
সমস্যাগুলি -> সমস্যা
যেমন -> যেমন
বহু -> বহু
মুখী -> মুখ
দেশের -> দেশ
বিরাট -> বিরাট
ছাত্রসমাজ -> ছাত্রসমাজ
তেমনি -> তেমন
বিভিন্ন -> বিভিন্ন
দলে -> দেওয়া
বিভক্ত -> বিভক্ত
হয়ে -> হওয়া
শহর -> শহর
গ্রামের -> গ্রাম
নানাপ্রকার -> নানাপ্রকার
সেবামূলক -> সেবামূলক
কাজে -> কাজ
অংশগ্রহণ -> অংশগ্রহণ
করতে -> করা
পারে -> পরা
যে -> যে
নিরক্ষরতা -> নিরক্ষর
উন্নয়নশীল -> উন্নয়নশীল
ভারতবর্ষের -> ভারতবর্ষ
সবচেয়ে -> সবচেয়ে
বড় -> বড়
অভিশাপ -> অভিশাপ
সমাজসেবার -> সমাজসেবা
অঙ্গরূপে -> অঙ্গরূপ
সেই -> সেই
নিরক্ষরতা -> নিরক্ষর
দূরীকরণে -> দূরীকরণ
ছাত্র -> ছাত্র
দের -> দের
প্রথম -> প্রথম
প্রধান -> প্রধান
কর্তব্য -> কর্তব্য
পল্লী -> পল্লী
প্রধান -> প্রধান
এই -> এই
দেশের -> দেশ
দ্বিতীয় -> দ্বিতীয়
বৃহত্তম -> বৃহত্তম
সমস্যা -> সমস্যা
পল্লীসংস্কার -> পল্লীসংস্কার
সুতরাং -> সুতরাং
সমাজসেবায় -> সমাজসেবা
বিষয় -> বিষয়
রূপে -> রূপ
ছাত্র -> ছাত্র
দের -> দের
পল্লী -> পল্লী
সংস্কারে -> সংস্কার
অংশগ্রহণ -> অংশগ্রহণ
করতে -> 

In [728]:
a='অভাবঅনটন'
print(list(a))

['অ', 'ভ', 'া', 'ব', 'অ', 'ন', 'ট', 'ন']
